# 交互式查看 cluster_map.tiff 中的 1024×1024 窗口

- 只按需读取一个 1024×1024 的小窗口，不加载整张 TIFF。
- 通过滑块选择窗口左上角坐标 (x, y)。
- 适用于 cluster label 图，使用离散 colormap (tab20)。


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import ipywidgets as widgets
from IPython.display import display

# 关闭 PIL 的大图像保护限制，仅在你能信任文件来源时这样做
Image.MAX_IMAGE_PIXELS = None

TIFF_PATH = "/nfs5/zyh/MUF-Clust/single_image_output/segmentation/11_Scan1/features/features_matrix_mean_cluster_cluster_map.tiff"
PATCH_SIZE = 1024

print("_____")
print("[INFO] 目标 tiff:", TIFF_PATH)
print("[INFO] PATCH_SIZE:", PATCH_SIZE)
print("_____")

if not os.path.exists(TIFF_PATH):
    print("_____")
    print("[ERROR] 文件不存在——请检查 TIFF_PATH")
    print("_____")
else:
    # 只用 PIL 读取图像尺寸，几乎不消耗时间和内存
    with Image.open(TIFF_PATH) as im:
        W, H = im.size

    max_x = max(0, W - PATCH_SIZE)
    max_y = max(0, H - PATCH_SIZE)

    print("[INFO] 图像尺寸 (W, H):", (W, H))
    print("[INFO] 滑块范围: x ∈ [0,", max_x, "] , y ∈ [0,", max_y, "]")

    x_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=max_x,
        step=max(1, PATCH_SIZE // 4),
        description="x",
        continuous_update=False,
    )
    y_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=max_y,
        step=max(1, PATCH_SIZE // 4),
        description="y",
        continuous_update=False,
    )

    out = widgets.Output()
    button = widgets.Button(description="显示 1024x1024 patch")

    def on_click(_):
        x = int(x_slider.value)
        y = int(y_slider.value)
        x = min(max(x, 0), max_x)
        y = min(max(y, 0), max_y)

        # 每次点击按钮时才真正从磁盘读取一个 1024x1024 的小块
        with Image.open(TIFF_PATH) as im:
            patch = im.crop((x, y, x + PATCH_SIZE, y + PATCH_SIZE))
            patch_np = np.array(patch)

        with out:
            out.clear_output(wait=True)
            plt.figure(figsize=(6, 6))
            plt.imshow(patch_np, cmap="tab20")
            plt.title(f"x={x}, y={y}, size={PATCH_SIZE}x{PATCH_SIZE}")
            plt.axis("off")
            plt.show()

    button.on_click(on_click)

    ui = widgets.VBox([x_slider, y_slider, button, out])
    display(ui)

_____
[INFO] 目标 tiff: /nfs5/zyh/MUF-Clust/single_image_output/segmentation/11_Scan1/features/features_matrix_mean_cluster_cluster_map.tiff
[INFO] PATCH_SIZE: 1024
_____
[INFO] 图像尺寸 (W, H): (34560, 50400)
[INFO] 滑块范围: x ∈ [0, 33536 ] , y ∈ [0, 49376 ]
